### Importing Modules

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
import os

### Load the dataset

In [13]:
data=pd.read_csv(r"U:\nlp_project\annclassification\artifacts\data\Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Preprocess the data
#### Drop irrelevant columns -> 'RowNumber', 'CustomerId', 'Surname'

In [14]:
data_copy = data.copy()

In [15]:
data_copy=data_copy.drop(['RowNumber','CustomerId','Surname'],axis=1)
data_copy

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


#### Encode categorical variables

- encoding gender:

In [17]:
label_encoder_gender=LabelEncoder()
data_copy['Gender']=label_encoder_gender.fit_transform(data_copy['Gender'])
data_copy

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


- Geography

In [19]:
onehot_encoder_geo=OneHotEncoder()

In [24]:
geo_encoder=onehot_encoder_geo.fit_transform(data_copy[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [25]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [26]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [27]:
## Combine one hot encoder columns with the original data
data_copy=pd.concat([data_copy.drop('Geography',axis=1),geo_encoded_df],axis=1)
data_copy.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


#### Save the encoders and scaler

- saving encoders:

In [49]:
encoders_dir = os.path.join(r'U:\nlp_project\annclassification\artifacts', 'encoders')
if not os.path.exists(encoders_dir):
    os.makedirs(encoders_dir)

encoders_path = os.path.join(encoders_dir, 'gender_label_encoder') + '.pkl'
with open(encoders_path, 'wb') as file:
    pickle.dump(label_encoder_gender, file)


In [104]:
encoders_dir = os.path.join(r'U:\nlp_project\annclassification\artifacts', 'encoders')
if not os.path.exists(encoders_dir):
    os.makedirs(encoders_dir)

encoders_path = os.path.join(encoders_dir, 'geo_one_hot_encoder') + '.pkl'
with open(encoders_path, 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)


##### Divide the dataset into indepent and dependent features

In [52]:
data_copy.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [60]:
X=data_copy.drop('Exited',axis=1)
y=data_copy['Exited']

# ## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# ## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [62]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [105]:
scaler_dir = os.path.join(r'U:\nlp_project\annclassification\artifacts', 'scaler')
if not os.path.exists(scaler_dir):
    os.makedirs(scaler_dir)

scaler_path = os.path.join(scaler_dir, 'scaler')
with open(scaler_path, 'wb') as file:
    pickle.dump(scaler, file)

In [47]:
# with open('scaler.pkl','wb') as file:
#     pickle.dump(scaler,file)

In [64]:
data_copy

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### ANN Implementation

In [65]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [66]:
(X_train.shape[1],)

(12,)

In [67]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
]

)

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [70]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [71]:
model

In [72]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [73]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [74]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.3902 - accuracy: 0.8390 - val_loss: 0.3503 - val_accuracy: 0.8545
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3549 - accuracy: 0.8550 - val_loss: 0.3434 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3486 - accuracy: 0.8576 - val_loss: 0.3566 - val_accuracy: 0.8505
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3442 - accuracy: 0.8596 - val_loss: 0.3445 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3435 - accuracy: 0.8570 - val_loss: 0.3388 - val_accuracy: 0.8645
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3405 - accuracy: 0.8626 - val_loss: 0.3379 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3353 - accuracy: 0.8579 - val_loss: 0.3465 - val_accuracy: 0.8580

In [81]:
type(history)

keras.src.callbacks.History

#### save model

In [79]:
model_dir = os.path.join(r'U:\nlp_project\annclassification\artifacts', 'model')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, 'model') + ".h5"
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

In [82]:
## Load Tensorboard Extension
%load_ext tensorboard

In [84]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 39440), started 0:01:48 ago. (Use '!kill 39440' to kill it.)

### Load the pickle file

In [97]:
artifacts_dir = r"U:\nlp_project\annclassification\artifacts\encoders"

label_encoder_gender_path = os.path.join(artifacts_dir, 'gender_label_encoder.pkl')
with open(label_encoder_gender_path, 'rb') as file:
    label_encoder_gender = pickle.load(file)

In [98]:
type(label_encoder_gender)

sklearn.preprocessing._label.LabelEncoder

In [101]:
type(geo_one_hot_encoder)

sklearn.preprocessing._label.LabelEncoder

In [103]:
type(model)

keras.src.engine.sequential.Sequential